# **Пайплайн для генерации речи с клонированием голоса на русском**
HF Модели: https://huggingface.co/Misha24-10/F5-TTS_RUSSIAN

Проблема была в том, что найти гайд на запуск, чтобы просто поиграть с моделькой, не так уж просто, есть гайд на запуск сервера с ней, с которого я и вырвал кусочки кода.

Гайд на дообучение мб когда то релизну

In [ ]:
! pip install f5-tts ruaccent

In [ ]:
from huggingface_hub import snapshot_download
import os

os.makedirs("./models/f5_tts_russian", exist_ok=True)

snapshot_download(
    repo_id="Misha24-10/F5-TTS_RUSSIAN",  
    local_dir="./models/f5_tts_russian",
    revision="main"
)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

F5TTS_v1_Base_v2/model_last.pt:   0%|          | 0.00/5.39G [00:00<?, ?B/s]

F5TTS_v1_Base_accent_tune/model_last.pt:   0%|          | 0.00/5.39G [00:00<?, ?B/s]

F5TTS_v1_Base/model_240000.pt:   0%|          | 0.00/5.39G [00:00<?, ?B/s]

F5TTS_v1_Base_accent_tune/model_10000_in(…):   0%|          | 0.00/1.35G [00:00<?, ?B/s]

F5TTS_v1_Base_v2/model_last_inference.sa(…):   0%|          | 0.00/1.35G [00:00<?, ?B/s]

F5TTS_v1_Base_accent_tune/model_20000_in(…):   0%|          | 0.00/1.35G [00:00<?, ?B/s]

F5TTS_v1_Base/model_240000_inference.saf(…):   0%|          | 0.00/1.35G [00:00<?, ?B/s]

F5TTS_v1_Base_accent_tune/model_last_inf(…):   0%|          | 0.00/1.35G [00:00<?, ?B/s]

'/content/models/f5_tts_russian'

In [ ]:
import os
import re
import time
import uuid
import glob
import subprocess
import requests
import torch
from ruaccent import RUAccent

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
INPUT_DIR = "/data/input"
OUTPUT_DIR = "/data/output"
MODEL_DIR = "/content/models/f5_tts_russian"

os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)


def process_with_ruaccent(text: str) -> str:
    """
    Обрабатывает текст с помощью RUAccent для постановки ударений
    """
    accentizer = RUAccent()
    accentizer.load(
        omograph_model_size='turbo3.1',
        use_dictionary=True,
        tiny_mode=False
    )
    return accentizer.process_all(text) + ' '

ckpt_path, vocab_path = "/content/models/f5_tts_russian/F5TTS_v1_Base_v2/model_last_inference.safetensors" ,"/content/models/f5_tts_russian/F5TTS_v1_Base/vocab.txt"


def generate_speech(
    text: str,
    ref_audio_path: str | None = None,
    ref_text: str | None = None,
    out_format: str = "wav",
    ckpt_path: str = ckpt_path,
    vocab_path: str = vocab_path,
    **kwargs
) -> str:
    """
    Генерирует речь из текста

    Args:
        text: Текст для синтеза
        ref_audio_path: Путь к референсному аудио (или URL)
        ref_text: Текст референсного аудио
        out_format: Формат выходного файла (wav/mp3)
        **kwargs: Дополнительные параметры для f5-tts_infer-cli

    Returns:
        Путь к сгенерированному аудиофайлу
    """
    gen_text = process_with_ruaccent(text)

    out_file = os.path.join(OUTPUT_DIR, f"out_{uuid.uuid4().hex}.wav")

    cmd = [
        "f5-tts_infer-cli",
        "--ckpt_file", ckpt_path,
        "--vocab_file", vocab_path,
        "--gen_text", gen_text,
        "--output_dir", OUTPUT_DIR,
        "--output_file", os.path.basename(out_file),
        "--device", DEVICE
    ]

    if ref_audio_path:
        cmd += ["--ref_audio", ref_audio_path]
    if ref_text:
        cmd += ["--ref_text", ref_text]

    for key, value in kwargs.items():
        if value is not None:
            cmd += [f"--{key}", str(value)]

    try:
        print("Запускаем генерацию речи...")
        print(" ".join(cmd))
        proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=600)
        if proc.returncode != 0:
            error_msg = proc.stderr.decode('utf-8')[:1000]
            raise Exception(f"f5-tts failed: {error_msg}")
        print("Генерация успешно завершена!")
    except subprocess.TimeoutExpired:
        raise Exception("TTS generation timed out")

    if not os.path.exists(out_file):
        wav_files = glob.glob(os.path.join(OUTPUT_DIR, "*.wav"))
        if wav_files:
            out_file = max(wav_files, key=os.path.getctime)
        else:
            raise Exception("No output audio file was generated")

    if out_format.lower() == "mp3":
        mp3_path = out_file.replace(".wav", ".mp3")
        subprocess.run(["ffmpeg", "-y", "-i", out_file, mp3_path],
                      stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return mp3_path

    return out_file

In [ ]:
# Для того чтоб аудио запустить
from IPython.display import Audio, display

# Опять же свой инпут, у меня вот такой тестовый вариант

inp = '''
Лекция по теме "Производные" для подготовки к ЕГЭ

Добрый день! Тема нашего урока — производные. Это очень важная и полезная тема в математике, которая обязательно пригодится вам на ЕГЭ. Не волнуйтесь, если вам кажется, что это сложно — мы разберем всё по шагам, с примерами, и вы поймете, что производные — это не страшно!
** Что такое производная? Интуитивное понимание **

Представьте, что вы едете на машине. Спидометр показывает вашу скорость. Но как определить скорость в конкретный момент времени? Это и есть основная идея производной.

Производная функции — это мгновенная скорость изменения функции в данной точке.

Математически: производная функции f(x) в точке x₀ — это предел отношения приращения функции к приращению аргумента при Δx → 0.

Но давайте объясним проще. Пусть у нас есть функция f(x). Возьмем точку x₀ и точку x₀ + Δx рядом с ней. Разница в значениях функции: Δf = f(x₀ + Δx) - f(x₀). Тогда отношение Δf/Δx показывает среднюю скорость изменения функции на интервале Δx. Производная — это предел этого отношения, когда Δx становится очень маленьким (стремится к нулю).

Обозначается производная так: f'(x) или df/dx.
2. Геометрический смысл производной

Это очень важно для ЕГЭ! Производная имеет наглядный геометрический смысл.

Геометрический смысл: Значение производной функции f(x) в точке x₀ равно угловому коэффициенту касательной к графику функции в этой точке.

Что такое угловой коэффициент? Это тангенс угла наклона прямой к оси OX.

Если f'(x₀) > 0, то касательная наклонена вверх (функция возрастает в этой точке).
Если f'(x₀) < 0, то касательная наклонена вниз (функция убывает в этой точке).
Если f'(x₀) = 0, то касательная горизонтальна (это может быть точка максимума, минимума или перегиба).
'''


ref_audio = '/content/alex_refer_cuted.mp3' # Загрузите свой и вставьте ссылку, рекомендую до 8-10 сек ато модель сойдет с ума
audio_path = generate_speech(
    text=inp,
    ref_audio_path=ref_audio,
    ref_text='Пирога. Веселящие чары слегка повыветрились и Гарри с Роном стали ощущать беспокойство', # Обязательно нужен текст референсного аудио, иначе модель сходит с ума
    out_format="mp3",
    speed=1, # Ну это скорость генерации
    cfg_strength=2 #  Сила управления генерацией через условие, от 1 до 5. 1 - модель генерит что хочет, 5 - роботизированный звук. Около 1.9-2.1 оптимальное значение
)

display(Audio(filename=audio_path))


Запускаем генерацию речи...
f5-tts_infer-cli --ckpt_file /content/models/f5_tts_russian/F5TTS_v1_Base_v2/model_last_inference.safetensors --vocab_file /content/models/f5_tts_russian/F5TTS_v1_Base/vocab.txt --gen_text 
Л+екция по т+еме Произв+одные для подгот+овки к ЕГ+Э

Д+обрый д+ень! Т+ема н+ашего ур+ока — произв+одные. +Это +очень в+ажная и пол+езная т+ема в матем+атике, кот+орая обяз+ательно пригод+ится в+ам на ЕГ+Э. Не волн+уйтесь, +если вам к+ажется, что +это сл+ожно — мы разбер+ём всё по шаг+ам, с прим+ерами, и в+ы пойм+ёте, что произв+одные — +это не стр+ашно!
 Чт+о так+ое произв+одная? Интуит+ивное поним+ание 

Предст+авьте, что вы +едете на маш+ине. Спид+ометр пок+азывает в+ашу ск+орость. Но к+ак определ+ить ск+орость в конкр+етный мом+ент вр+емени? +Это и +есть основн+ая ид+ея произв+одной.

Произв+одная ф+ункции — +это мгнов+енная ск+орость измен+ения ф+ункции в д+анной т+очке.

Математ+ически: произв+одная ф+ункции f(x) в т+очке x — +это пред+ел отнош+ения приращ+ения ф+ун